#### Pool

In [1]:
from multiprocessing import Pool

def f(x):
    return x * x

if __name__ == '__main__':
    with Pool(processes=5) as p:
        print(p.map(f, [1, 2, 3]))

[1, 4, 9]


#### Process

In [2]:
from multiprocessing import Process

def f(name):
    print('hello ', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob', ))
    p.start()
    p.join()

hello  bob


In [3]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name: ', __name__)
    print('parent process: ', os.getppid())
    print('process id: ', os.getpid())

def f(name):
    info('function f')
    print('hello, ', name)


if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('boby', ))
    p.start()
    p.join()

main line
module name:  __main__
parent process:  331474
process id:  332384
function f
module name:  __main__
parent process:  332384
process id:  332753
hello,  boby


##### something using lock

In [4]:
from multiprocessing import Process, Queue

def f(q):
    q.put('X' * 1000)

if __name__ == '__main__':
    queue = Queue()
    p = Process(target=f, args=(queue, ))
    p.start()
    p.join()
    obj = queue.get()

In [5]:
from multiprocessing import Process, Lock

def f(x):
    res = x ** 3
    print(res)

if __name__ == '__main__':
    lock = Lock()
    for i in range(10):
        Process(target=f, args=(i, )).start()

0
1
8
27
64
125
216
343
512
729


In [ ]:
from multiprocessing import Process, Lock

def f(l, x):
    print(f'hello world {x}')


if __name__ == '__main__':
    lock = Lock()
    for i in range(10):
        Process(target=f, args=(lock, i,)).start()

##### Pickle

In [ ]:
import pickle

data = {'ex': [1, 2, 3, 4, 5]}
serialized_data = pickle.dump(data)

# unpickling
unpickled_data = pickle.loads(serialized_data)

unpickled_data

In [58]:
from multiprocessing import Process, freeze_support, set_start_method

def f():
    print('hello')

if __name__ == '__main__':
    freeze_support()
    # set_start_method('spawn')
    p = Process(target=f)
    p.start()

hello


#### Context and start methods ex

In [63]:
import multiprocessing as mp

def f(q):
    q.put('hello_')

if __name__ == '__main__':
    # mp.set_start_method('forkserver')
    q = mp.Queue()
    p = mp.Process(target=f, args=(q, ))
    p.start()
    print(q.get())
    p.join()

hello_


In [ ]:
import multiprocessing as mp

def f(q):
    q.put('hello_')

if __name__ == '__main__':
    ctx = mp.get_context('forkserver')
    q = ctx.Queue()
    p = ctx.Process(target=f, args=(q, ))
    p.start()
    print(q.get())
    p.join()

communication channel between processes.
<br>Queues are thread and process safe.


In [66]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q, ))
    p.start()
    print(q.get())
    p.join()

[42, None, 'hello']


Pipes

In [67]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'he'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn, ))
    p.start()
    print(parent_conn.recv())
    p.join()

[42, None, 'he']


Synchronization between process

In [73]:
from multiprocessing import Process, Lock

def f(lock, i):
    lock.acquire()
    try:
        print('hello world', i)
    finally:
        lock.release

if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        p = Process(target=f, args=(lock, num))
        p.start()

hello world 0


Shared memory between process

In [74]:
from multiprocessing import Process, Value, Array

def f(n, a):
    n.value = 3.1416
    for i in range(len(a)):
        a[i] = -a[i]

if __name__ == '__main__':
    num = Value('d', 0.0)
    arr = Array('i', range(10))

    p = Process(target=f, args=(num, arr))
    p.start()
    p.join()

    print(num.value)
    print(arr[:])

3.1416
[0, -1, -2, -3, -4, -5, -6, -7, -8, -9]


Pool of workers

In [76]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x ** 2

if __name__ == '__main__':
    with Pool(processes=4) as pool:
        
        # print [0, 1, 4, ...., 81]
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)
        
        # evaluate 'f(20)' asynchronously
        res = pool.apply_async(f, (20, ))   # runs only 1 process
        print(res.get(timeout=1))

        # evalueate 'os.getpid()' asynchronously
        res = pool.apply_async(os.getpid, ())
        print(res.get(timeout=1))

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 seconds
        res = pool.apply_async(time.sleep, (10, ))

        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing.TimeoutError")

        print("For the moment, the pool remains available for more work")

  # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")






[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
9
4
25
16
49
64
81
36
400
23980
[23978, 23981, 23979, 23980]
We lacked patience and got a multiprocessing.TimeoutError
For the moment, the pool remains available for more work
Now the pool is closed and no longer available


Ex

In [ ]:
from multiprocessing import Process

p = Process(target=print, args=(1, ))
p.run()